# NeuralPLSI vs PLSI Tutorial

This notebook demonstrates the usage of `NeuralPLSI` and `PLSI` (Spline-based) models on simulated data.

**Scenario:**
- **N**: 500
- **Outcome**: Continuous
- **g-function**: Sigmoid
- **Exposure Distribution**: Normal

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from models.PLSI import SplinePLSI
from models.nPLSI import neuralPLSI
from simulation import simulate_data

sns.set_style("whitegrid")
%matplotlib inline

## 1. Data Generation

We generate synthetic data where the outcome $Y$ depends on exposures $X$ through a single index $\beta^T X$ transformed by a nonlinear function $g(\cdot)$, plus linear covariate effects $Z\gamma$.

In [ ]:
n = 500
outcome = 'continuous'
g_fn_name = 'sigmoid'
seed = 42

# Generate data
X, Z, y, xb, gxb, true_g_fn = simulate_data(
    n=n, 
    outcome=outcome, 
    g_type=g_fn_name, 
    seed=seed
)

print(f"X shape: {X.shape}")
print(f"Z shape: {Z.shape}")
print(f"y shape: {y.shape}")

## 2. Model Fitting

We fit both the standard Spline-based PLSI and the NeuralPLSI model.

In [ ]:
# Initialize models
plsi = SplinePLSI(family=outcome)
nplsi = neuralPLSI(family=outcome)

# Fit PLSI
print("Fitting PLSI...")
plsi.fit(X, Z, y)

# Fit NeuralPLSI
print("Fitting NeuralPLSI...")
nplsi.fit(X, Z, y)

## 3. Evaluation and Visualization

We compare the estimated $g(\cdot)$ functions against the true function.

In [ ]:
# Create a grid for plotting
grid = np.linspace(xb.min(), xb.max(), 100)
true_g_vals = np.vectorize(true_g_fn)(grid)

# Predict g(index) for the grid
plsi_g_vals = plsi.g_function(grid)
nplsi_g_vals = nplsi.g_function(grid)

# Plot
plt.figure(figsize=(10, 6))
plt.plot(grid, true_g_vals, 'k--', label='True g(x)', linewidth=2)
plt.plot(grid, plsi_g_vals, 'b-', label='PLSI', alpha=0.7)
plt.plot(grid, nplsi_g_vals, 'r-', label='NeuralPLSI', alpha=0.7)

plt.title(f"Estimated vs True g(x) ({g_fn_name})", fontsize=14)
plt.xlabel("Index (Beta^T X)")
plt.ylabel("g(Index)")
plt.legend()
plt.show()

### Parameter Estimation

Comparison of the estimated beta coefficients.

In [ ]:
from simulation import beta as true_beta

plt.figure(figsize=(10, 5))
indices = np.arange(len(true_beta))
width = 0.25

plt.bar(indices - width, true_beta, width, label='True', color='black', alpha=0.6)
plt.bar(indices, plsi.beta, width, label='PLSI', color='blue', alpha=0.6)
plt.bar(indices + width, nplsi.beta, width, label='NeuralPLSI', color='red', alpha=0.6)

plt.xlabel("Beta Index")
plt.ylabel("Value")
plt.title("Beta Coefficient Comparison")
plt.legend()
plt.show()